In [ ]:
#DESCARGAR CONEXION A MYSQL

In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# importar csv a mysql

In [3]:
import json
import mysql.connector
from mysql.connector import Error
import pandas as pd 

DB_CONFIG = {
    'host': '127.0.0.1',
    'database': 'wine', 
    'user': 'root',   
    'password': '2112'  
}

# Ruta 
CSV_FILE_PATH = 'C:/Users/USER/OneDrive - Escuela Politécnica Nacional/Escritorio/Análisis de Datos/ProyectoFinalADD/winemag_data_first150k.csv' 

def import_wine_csv_data():
    conn = None
    try:
        # Conectar a la base de datos MySQL
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Cargar el archivo CSV en un DataFrame de Pandas
        df = pd.read_csv(CSV_FILE_PATH, index_col=0)
        df.index.name = 'id'
        df.reset_index(inplace=True)

        print(f"Total de reseñas de vino a importar desde CSV: {len(df)} filas.")

        # SQL para insertar datos
        insert_sql = """
        INSERT INTO wine_reviews_csv (
            id, country, description, designation, points, price, province,
            region_1, region_2, variety, winery
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """

        for i, row in enumerate(df.itertuples(index=False)):
            values = (
                row.id,
                row.country,
                row.description,
                row.designation if pd.notna(row.designation) else None,
                row.points,
                row.price if pd.notna(row.price) else None,
                row.province,
                row.region_1 if pd.notna(row.region_1) else None,
                row.region_2 if pd.notna(row.region_2) else None,
                row.variety,
                row.winery
            )
            try:
                cursor.execute(insert_sql, values)
                if (i + 1) % 10000 == 0:
                    print(f"  Insertados {i + 1} registros...")
            except Error as err:
                print(f"Error al insertar la reseña {i+1} (ID: {row.id}): {err}")
                pass

        conn.commit()
        print("\nImportación de reseñas de vino desde CSV completada exitosamente.")

    except FileNotFoundError:
        print(f"Error: El archivo '{CSV_FILE_PATH}' no se encontró.")
    except pd.errors.EmptyDataError:
        print(f"Error: El archivo CSV '{CSV_FILE_PATH}' está vacío.")
    except pd.errors.ParserError as e:
        print(f"Error al parsear el CSV: {e}")
    except Error as e:
        print(f"Error de MySQL: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()
            print("Conexión a MySQL cerrada.")

if __name__ == "__main__":
    import_wine_csv_data()

Total de reseñas de vino a importar desde CSV: 150930 filas.
Error al insertar la reseña 1134 (ID: 1133): 1054 (42S22): Unknown column 'nan' in 'field list'
  Datos problemáticos: (1133, nan, 'Delicate white flowers and a spin of lemon peel on the nose start this refined white. The bright fruit on the palate is tropical and exotic, but the minerality gives it lift. Fuller-bodied but poised, the wine has aging potential and a food-friendly character.', 'Askitikos', 90, 17.0, nan, None, None, 'Assyrtiko', 'Tsililis')
Error al insertar la reseña 1441 (ID: 1440): 1054 (42S22): Unknown column 'nan' in 'field list'
  Datos problemáticos: (1440, nan, 'A blend of 60% Syrah, 30% Cabernet Sauvignon and 10% Merlot, this inky garnet-colored wine offers aromas of cassis and elderberry. On the palate, there is a combination of cooked fruit and cool spice flavors yet there is no lack of accompanying acidity. Flavors of cassis, elderberry, anise, orange peel and vanilla are backed by silky tannins tha

In [ ]:
#IMPORTAR JSON A MYSQL

In [10]:
import json
import mysql.connector
from mysql.connector import Error

# configuración de mysql
DB_CONFIG = {
    'host': '127.0.0.1',      
    'database': 'wine', 
    'user': 'root',  
    'password': '2112' 
}

# ruta
JSON_FILE_PATH = 'C:/Users/USER/OneDrive - Escuela Politécnica Nacional/Escritorio/Análisis de Datos/ProyectoFinalADD/winemag_data_130k_v2.json'

def import_wine_data():
    conn = None
    try:
        # Conectar a la base de datos MySQL
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, list): 
            print(f"Total de reseñas de vino a importar: {len(data)}")

            insert_sql = """
            INSERT INTO wine_reviews_json (
                points, title, description, taster_name, taster_twitter_handle,
                price, designation, variety, region_1, region_2, province, country, winery
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """

            for i, review in enumerate(data):

                values = (
                    int(review.get('points')) if review.get('points') is not None else None,
                    review.get('title'),
                    review.get('description'),
                    review.get('taster_name'),
                    review.get('taster_twitter_handle'),
                    float(review.get('price')) if review.get('price') is not None else None,
                    review.get('designation'),
                    review.get('variety'),
                    review.get('region_1'),
                    review.get('region_2'),
                    review.get('province'),
                    review.get('country'),
                    review.get('winery')
                )
                try:
                    cursor.execute(insert_sql, values)
                except Error as err:
                    print(f"Error al insertar la reseña {i+1}: {err}")
                    print(f"  Datos problemáticos: {values}")
                    pass ¿

            # Confirmar los cambios en la base de datos
            conn.commit()
            print("Importación de reseñas de vino completada exitosamente.")
        else:
            print("El archivo JSON no es una lista de objetos, o tiene un formato inesperado.")

    except FileNotFoundError:
        print(f"Error: El archivo '{JSON_FILE_PATH}' no se encontró.")
    except json.JSONDecodeError as e:
        print(f"Error al decodificar el JSON: {e}")
    except Error as e:
        print(f"Error de MySQL: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()
            print("Conexión a MySQL cerrada.")

if __name__ == "__main__":
    import_wine_data()

Total de reseñas de vino a importar: 129971
Importación de reseñas de vino completada exitosamente.
Conexión a MySQL cerrada.


In [ ]:
#IMPORTAR LA DATABASE EN MYSQL A PYTHON

In [6]:
import mysql.connector
import pandas as pd
from mysql.connector import Error

# conexión a MySQL 
DB_CONFIG = {
    'host': '127.0.0.1',
    'database': 'wine',
    'user': 'root',
    'password': '2112'
}

def export_mysql_to_pandas(table_name): 
    conn = None
    df = None
    try:
        # Conectar a la base de datos MySQL
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        query = f"SELECT * FROM {table_name}" 
        print(f"Ejecutando consulta: {query}")

        df = pd.read_sql_query(query, conn) 

        print(f"Datos de la tabla '{table_name}' cargados exitosamente en un DataFrame. Filas: {len(df)}")
        return df

    except Error as e:
        print(f"Error al conectar o ejecutar la consulta en MySQL: {e}")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return None
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()
            print("Conexión a MySQL cerrada.")

if __name__ == "__main__":
    print("--- Importando datos de wine_reviews_json a Python ---")
    df_json = export_mysql_to_pandas('wine_reviews_json') 
    if df_json is not None:
        print("Primeras 5 filas del DataFrame de JSON:")
        print(df_json.head())
        print("\nInformación del DataFrame de JSON:")
        df_json.info()

    print("\n" + "="*50 + "\n")

    print("--- Importando datos de wine_reviews_csv a Python ---")
    df_csv = export_mysql_to_pandas('wine_reviews_csv') 
    if df_csv is not None:
        print("Primeras 5 filas del DataFrame de CSV:")
        print(df_csv.head())
        print("\nInformación del DataFrame de CSV:")
        df_csv.info()

--- Importando datos de wine_reviews_json a Python ---
Ejecutando consulta: SELECT * FROM wine_reviews_json


C:\Users\USER\AppData\Local\Temp\ipykernel_25208\267908590.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Datos de la tabla 'wine_reviews_json' cargados exitosamente en un DataFrame. Filas: 259942
Conexión a MySQL cerrada.
Primeras 5 filas del DataFrame de JSON:
   id  points                                              title  \
0   1      87                  Nicosia 2013 Vulkà Bianco  (Etna)   
1   2      87      Quinta dos Avidagos 2011 Avidagos Red (Douro)   
2   3      87      Rainstorm 2013 Pinot Gris (Willamette Valley)   
3   4      87  St. Julian 2013 Reserve Late Harvest Riesling ...   
4   5      87  Sweet Cheeks 2012 Vintner's Reserve Wild Child...   

                                         description         taster_name  \
0  Aromas include tropical fruit, broom, brimston...       Kerin O’Keefe   
1  This is ripe and fruity, a wine that is smooth...          Roger Voss   
2  Tart and snappy, the flavors of lime flesh and...        Paul Gregutt   
3  Pineapple rind, lemon pith and orange blossom ...  Alexander Peartree   
4  Much like the regular bottling from 2012, this...  

C:\Users\USER\AppData\Local\Temp\ipykernel_25208\267908590.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Datos de la tabla 'wine_reviews_csv' cargados exitosamente en un DataFrame. Filas: 150925
Conexión a MySQL cerrada.
Primeras 5 filas del DataFrame de CSV:
   id country                                        description  \
0   0      US  This tremendous 100% varietal wine hails from ...   
1   1   Spain  Ripe aromas of fig, blackberry and cassis are ...   
2   2      US  Mac Watson honors the memory of a wine once ma...   
3   3      US  This spent 20 months in 30% new French oak, an...   
4   4  France  This is the top wine from La Bégude, named aft...   

                            designation  points  price        province  \
0                     Martha's Vineyard      96  235.0      California   
1  Carodorum Selección Especial Reserva      96  110.0  Northern Spain   
2         Special Selected Late Harvest      96   90.0      California   
3                               Reserve      96   65.0          Oregon   
4                            La Brûlade      95   66.0        Prov

In [7]:
# UNIFICACION DE DATOS

In [8]:
import pandas as pd

print("Iniciando la unificación de DataFrames...")

df_unified = pd.concat([df_json, df_csv], ignore_index=True)

print("Unificación completada.")
print(f"Número total de reseñas unificadas: {len(df_unified)} filas.")

# verificar
print("\nPrimeras 5 filas del DataFrame unificado:")
print(df_unified.head())

print("\nÚltimas 5 filas del DataFrame unificado:")
print(df_unified.tail())

print("\nInformación del DataFrame unificado:")
df_unified.info()

print("\nColumnas del DataFrame unificado:")
print(df_unified.columns)

Iniciando la unificación de DataFrames...
Unificación completada.
Número total de reseñas unificadas: 410867 filas.

Primeras 5 filas del DataFrame unificado:
   id  points                                              title  \
0   1      87                  Nicosia 2013 Vulkà Bianco  (Etna)   
1   2      87      Quinta dos Avidagos 2011 Avidagos Red (Douro)   
2   3      87      Rainstorm 2013 Pinot Gris (Willamette Valley)   
3   4      87  St. Julian 2013 Reserve Late Harvest Riesling ...   
4   5      87  Sweet Cheeks 2012 Vintner's Reserve Wild Child...   

                                         description         taster_name  \
0  Aromas include tropical fruit, broom, brimston...       Kerin O’Keefe   
1  This is ripe and fruity, a wine that is smooth...          Roger Voss   
2  Tart and snappy, the flavors of lime flesh and...        Paul Gregutt   
3  Pineapple rind, lemon pith and orange blossom ...  Alexander Peartree   
4  Much like the regular bottling from 2012, this...

In [9]:
#limpieza de datos

In [16]:
import pandas as pd

if 'df_unified' not in locals():
    print("Error, no está definido 'df_uniefied'")
else:
    print("Iniciando limpieza de datos")
    print(f"Número de filas iniciales: {len(df_unified)}")

    # transformar tipos de datos
    # 'points' y 'price' a valores numericos
    # errors='coerce' convierte cualquier valor no numérico a NaN
    print(" ::::::::::::::::::::::::::: Transformar tipos de datos :::::::::::::::::::::::::::::::::::")
    df_unified['points'] = pd.to_numeric(df_unified['points'], errors='coerce')
    df_unified['price'] = pd.to_numeric(df_unified['price'], errors='coerce')

    print("\nTipos de datos después de la conversión:")
    print(df_unified[['points', 'price']].info())

    
    # VALORES NULOS (NaN) EN COLUMNAS DE TEXTO 
    # Rellenar los valores NaN con un string descriptivo
    print(" ::::::::::::::::::::::::::: Valores nulos :::::::::::::::::::::::::::::::::::")
    for col in ['country', 'description', 'designation', 'province', 'region_1', 'region_2', 'variety', 'winery', 'title', 'taster_name', 'taster_twitter_handle']:
        if col in df_unified.columns:
            df_unified[col] = df_unified[col].fillna('desconocido')

    print("\nValores nulos después de rellenar (deberían ser 0 para las columnas de texto):")
    print(df_unified.isnull().sum())


    # ELIMINAR DUPLICADOS 
    
    print("::::::::::::::::::::::::::::::::::::: Eliminar registros duplicados ::::::::::::::::::::::")
    num_duplicates_initial = df_unified.duplicated(subset=['description', 'winery', 'points', 'price']).sum()
    print(f"Número de duplicados encontrados: {num_duplicates_initial}")

    df_unified.drop_duplicates(subset=['description', 'winery', 'points', 'price'], inplace=True)
    
    print(f"Número de filas finales después de eliminar duplicados: {len(df_unified)}")


    # verificar
    print("\nEstado final del DataFrame después de la limpieza:")
    df_unified.info()

    print("\nEstadísticas del DataFrame después de la limpieza:")
    print(df_unified.describe(include='all'))

    print("\nLimpieza completa")

Iniciando limpieza de datos
Número de filas iniciales: 169652
 ::::::::::::::::::::::::::: Transformar tipos de datos :::::::::::::::::::::::::::::::::::

Tipos de datos después de la conversión:
<class 'pandas.core.frame.DataFrame'>
Index: 169652 entries, 0 to 409576
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   points  169652 non-null  int64  
 1   price   156817 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.9 MB
None
 ::::::::::::::::::::::::::: Valores nulos :::::::::::::::::::::::::::::::::::

Valores nulos después de rellenar (deberían ser 0 para las columnas de texto):
id                           0
points                       0
title                        0
description                  0
taster_name                  0
taster_twitter_handle        0
price                    12835
designation                  0
variety                      0
region_1                     0
region_2               

In [3]:
#Analisis de sentimiento

In [1]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

print("Empezando análisis de sentimiento")
# Crear la instancia del analizador
analyzer = SentimentIntensityAnalyzer()

# función para obtener la puntuación de sentimiento
def get_sentiment_score(text):
    if pd.isna(text): 
        return 0.0
    vs = analyzer.polarity_scores(text)
    return vs['compound']

# aplicar la función a la columna 'description' del DataFrame
# columna'sentiment_score'
df_unified['sentiment_score'] = df_unified['description'].apply(get_sentiment_score)

print("Análisis de sentimiento finalizado")

# resultado
# cerca de 1: Muy positivo ::::: cerca de -1: Muy negativo ::::: cerca de 0: Neutro

# reseñas positivas
print("\nReseñas más positivas (top 12):")
top_positive = df_unified.sort_values(by='sentiment_score', ascending=False).head(12)
print(top_positive[['description', 'sentiment_score']])

# reseñas negativas
print("\nReseñas más negativas (top 12):")
top_negative = df_unified.sort_values(by='sentiment_score', ascending=True).head(12)
print(top_negative[['description', 'sentiment_score']])

# estadistica del analisis sentimental
print("\nEstadísticas del sentimiento general:")
print(df_unified['sentiment_score'].describe())

Iniciando el análisis de sentimiento...
Análisis de sentimiento completado.

Reseñas más positivas (top 5):
                                              description  sentiment_score
123946  There is not just a notion of ripeness but alm...           0.9937
325952  This is a splendid wine. Yes, it is more power...           0.9931
375502  This may be the most expensive Malbec in Calif...           0.9926
323147  This brilliant PG has just a bit of older Fren...           0.9917
68665   A touch of flinty reduction clings to the nose...           0.9912

Reseñas más negativas (top 5):
                                              description  sentiment_score
82547   The vintage was terribly difficult, with cold ...          -0.9288
368615  Disappointing considering the price. There's n...          -0.9202
101307  This is an underripe stench bomb with pyrazine...          -0.9153
306548  Hard, leathery aromas, with an intense but exc...          -0.9062
29581   This is a wine the winery h

In [17]:
import pandas as pd

print("Guardar el dataframe a csv")

output_file = 'unifed_wine_reviews_limpio.csv'

df_unified.to_csv(output_file, index=False, encoding='utf-8')

print(f"DataFrame guardado exitosamente en '{output_file}'.")

Guardar el dataframe a csv
DataFrame guardado exitosamente en 'unifed_wine_reviews_limpio.csv'.


In [18]:
import pandas as pd

if 'df_unified' in locals():
    print(f"Número de filas: {df_unified.shape[0]}")
else:
    print("Error")

Número de filas: 169652
